<a href="https://colab.research.google.com/github/suyash-jamdade/Recommendation-systems/blob/master/Collaborative_Filtering_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-04-10 13:13:55--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  34.3MB/s    in 4.4s    

2020-04-10 13:14:00 (34.8 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [0]:
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [18]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [20]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [0]:
movies_df = movies_df.drop('genres',1)

In [22]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [23]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [0]:
ratings_df = ratings_df.drop('timestamp', 1)

In [25]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [26]:
userInput = [
             {'title' : 'Breakfast Club ,The','rating':5},
             {'title' : 'Toy Story','rating':3.5},
             {'title' : 'Jumanji','rating':2},
             {'title' : 'Pulp Fiction','rating':5},
             {'title' : 'Akira','rating':4.5}
]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club ,The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [31]:
#filtering
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#merging
inputMovies = pd.merge(inputId, inputMovies)
#droping
inputMovies = inputMovies.drop('year',1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5


In [33]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
749,15,1,4.0


In [0]:
userSubsetGroup = userSubset.groupby(['userId'])

In [36]:
userSubsetGroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5


In [0]:
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)

In [39]:
userSubsetGroup[0:3]

[(75,       userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5), (106,       userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0), (217,        userId  movieId  rating
  19726     217        1     4.0
  19727     217        2     2.0
  19751     217      296     5.0
  19812     217     1274     3.5)]

In [0]:
userSubsetGroup = userSubsetGroup[0:100]


In [0]:
#Pearson Correlation

pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
  group = group.sort_values(by = 'movieId')
  inputMovies = inputMovies.sort_values(by='movieId')

  nRatings = len(group)
  temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
  tempRatingList = temp_df['rating'].tolist()
  tempGroupList = group['rating'].tolist()
  Sxx = sum([i**2 for i in tempRatingList])-pow(sum(tempRatingList),2)/float(nRatings)
  Syy = sum([i**2 for i in tempGroupList])-pow(sum(tempGroupList),2)/float(nRatings)
  Sxy = sum( i*j for i,j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

  if Sxx != 0 and Syy != 0:
    pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
  else:
    pearsonCorrelationDict[name] = 0

In [46]:
pearsonCorrelationDict.items()

dict_items([(75, 0.8017837257372732), (106, 0.4629100498862757), (217, 0.8819171036881969), (393, 0.592156525463792), (686, 0.8819171036881969), (815, 0.5194624816493197), (1040, 0.9456108576893003), (1066, 0.9650595721775965), (1130, 0.17039954414774924), (1414, 0.90694099390581), (1502, 0.8510644963469901), (1599, 0.629940788348712), (1625, 0.8510644963469901), (1824, 0.8017837257372732), (1950, 0.4364357804719848), (1966, 0.8819171036881969), (2065, 0.629940788348712), (2128, 0.4795122238161564), (2432, 0.1543033499620919), (2473, 0.7237468644557459), (2726, 0.7715167498104595), (2791, 0.8819171036881969), (2839, 0.9869275424396534), (2948, 0.4364357804719848), (3025, 0.3563483225498992), (3040, 0.8728715609439696), (3051, 0.9341484842923421), (3116, 0.7122123116119197), (3186, 0.6351073488299558), (3271, 0.24209101306752098), (3388, 0.5509080804800992), (3392, -0.3779644730092272), (3429, 0.2182178902359924), (3734, -0.3563483225498992), (4072, 0.9869275424396534), (4079, 0.8861418

In [50]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.801784,75
1,0.462910,106
2,0.881917,217
3,0.592157,393
4,0.881917,686


In [51]:
topUsers = pearsonDF.sort_values(by='similarityIndex',ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
92,0.991460,9721
59,0.991460,6157
60,0.986928,6207
57,0.986928,6082
22,0.986928,2839


In [52]:
topUserRating = topUsers.merge(ratings_df, left_on='userId', right_on='userId',how='inner')
topUserRating.head()

,similarityIndex,userId,movieId,rating
0,0.99146,9721,1,3.5
1,0.99146,9721,2,2.0
2,0.99146,9721,6,3.0
3,0.99146,9721,7,1.0
4,0.99146,9721,10,1.0


In [55]:
topUserRating['weightedRating'] = topUserRating['similarityIndex']*topUserRating['rating']
topUserRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.99146,9721,1,3.5,3.47011
1,0.99146,9721,2,2.0,1.98292
2,0.99146,9721,6,3.0,2.97438
3,0.99146,9721,7,1.0,0.99146
4,0.99146,9721,10,1.0,0.99146


In [56]:
tempTopUsersRating = topUserRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,45.013044,166.009785
2,45.013044,112.784487
3,8.448792,24.466262
4,0.801784,1.603567
5,6.333896,15.508922


In [58]:
recommendation_df = pd.DataFrame()

recommendation_df['weight average recommendation score'] = tempTopUsersRating['sum_similarityIndex']/tempTopUsersRating['sum_weightedRating']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weight average recommendation score,movieId
movieId,,
1,0.271147,1
2,0.399107,2
3,0.345324,3
4,0.500000,4
5,0.408403,5


In [59]:
recommendation_df = recommendation_df.sort_values(by='weight average recommendation score', ascending=False)
recommendation_df.head(10)

,weight average recommendation score,movieId
movieId,,
5675,2.0,5675
43836,2.0,43836
1739,2.0,1739
6827,2.0,6827
103695,2.0,103695
6948,2.0,6948
4450,2.0,4450
4371,2.0,4371
62912,2.0,62912


In [60]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
1677,1739,3 Ninjas: High Noon On Mega Mountain,1998
4277,4371,Baby Boy,2001
4356,4450,Bully,2001
5577,5675,Swept Away,2002
6718,6827,It's Pat,1994
6837,6948,Tupac: Resurrection,2003
10850,43836,"Pink Panther, The",2006
11085,45981,Hollow Man II,2006
13104,62912,High School Musical 3: Senior Year,2008
21421,103695,"Starfighters, The",1964
